In [1]:
#import the required libriary used for the analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import json as js
import ast

In [2]:
#open file and handle the issue of single quotes
with open(r"W:\covid\places.txt", encoding="utf8") as corona_file:
    pl=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        pl.append(temp)

    pls_df=pd.DataFrame(pl)
    print(pls_df.head())

      country country_code              full_name  \
0   Allemagne           DE  Düsseldorf, Allemagne   
1  Etats-Unis           US          Manhattan, NY   
2  Etats-Unis           US       Connecticut, USA   
3        Inde           IN        नई दिल्ली, भारत   
4  Etats-Unis           US        Albuquerque, NM   

                                                 geo                id  \
0  {'type': 'Feature', 'bbox': [6.688728, 51.1243...  b7d3c12268abd20e   
1  {'type': 'Feature', 'bbox': [-74.026675, 40.68...  01a9a39529b27f36   
2  {'type': 'Feature', 'bbox': [-73.727776, 40.95...  e86b380cfefcced5   
3  {'type': 'Feature', 'bbox': [76.84252, 28.3976...  317fcc4b21a604d5   
4  {'type': 'Feature', 'bbox': [-106.7916912, 35....  813a485b26b8dae2   

          name place_type  
0   Düsseldorf       city  
1    Manhattan       city  
2  Connecticut      admin  
3    नई दिल्ली       city  
4  Albuquerque       city  


In [3]:
#helper function to calculate the average and apply them to all
def average(number1, number2):
    return (number1 + number2) / 2.0

#compute the average data of latitude and lontitude 
pls_df['Lontitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][0],x['geo']['bbox'][2]),axis = 1)
pls_df['Latitude'] = pls_df.apply(lambda x: average (x['geo']['bbox'][1],x['geo']['bbox'][3]),axis = 1)
pls_df

,country,country_code,full_name,geo,id,name,place_type,Lontitude,Latitude
0,Allemagne,DE,"Düsseldorf, Allemagne","{'type': 'Feature', 'bbox': [6.688728, 51.1243...",b7d3c12268abd20e,Düsseldorf,city,6.814308,51.238447
1,Etats-Unis,US,"Manhattan, NY","{'type': 'Feature', 'bbox': [-74.026675, 40.68...",01a9a39529b27f36,Manhattan,city,-73.968542,40.780709
2,Etats-Unis,US,"Connecticut, USA","{'type': 'Feature', 'bbox': [-73.727776, 40.95...",e86b380cfefcced5,Connecticut,admin,-72.757385,41.500753
3,Inde,IN,"नई दिल्ली, भारत","{'type': 'Feature', 'bbox': [76.84252, 28.3976...",317fcc4b21a604d5,नई दिल्ली,city,77.095086,28.638489
4,Etats-Unis,US,"Albuquerque, NM","{'type': 'Feature', 'bbox': [-106.7916912, 35....",813a485b26b8dae2,Albuquerque,city,-106.632718,35.117003
5,Etats-Unis,US,"Kansas, USA","{'type': 'Feature', 'bbox': [-102.051769, 36.9...",27c45d804c777999,Kansas,admin,-98.319925,38.498196
6,Etats-Unis,US,"Laguna, CA","{'type': 'Feature', 'bbox': [-121.4882357, 38....",947299911720444c,Laguna,city,-121.439383,38.410428
7,Etats-Unis,US,"New Braunfels, TX","{'type': 'Feature', 'bbox': [-98.2149552, 29.6...",c807df9bb77ea0b3,New Braunfels,city,-98.123250,29.705614
8,Etats-Unis,US,"DeWitt, MI","{'type': 'Feature', 'bbox': [-84.6117479, 42.7...",44058dee09d58997,DeWitt,city,-84.572933,42.827637
9,Etats-Unis,US,"Texas, USA","{'type': 'Feature', 'bbox': [-106.645646, 25.8...",e0060cda70f5f341,Texas,admin,-100.076888,31.168893


In [4]:
#open file and handle the issue of single quotes
with open(r"W:\covid\corona_tweets_50_data.txt", encoding="utf8") as corona_file:
    tw=[]
    line= corona_file.readlines()
    for row in range(len(line)):
        temp = ast.literal_eval(line[row])
        tw.append(temp)
    tws_df=pd.DataFrame(tw)

In [5]:
def geo_nan(df):
    if isinstance(df.loc['geo'], float):
        return False
    if 'place_id' in df['geo']:
        return True
    return False

tws_df['geo_nan'] = tws_df.apply(geo_nan, axis=1)
tws_df = tws_df[tws_df['geo_nan'] == True]

In [6]:
tws_df['geo_id'] = tws_df.apply(lambda x: x['geo']['place_id'],axis = 1)
tws_merge = pd.merge(tws_df,pls_df,how = 'inner',left_on = 'geo_id',right_on = 'id')

In [7]:
from datetime import datetime
#define the helper function to do the function take out the string of time and summarize them into the hour
def process_time(df):
    time_str = df['created_at'][11:19]
    time_whole = datetime.strptime(time_str, '%H:%M:%S')
    return int(datetime.strftime(time_whole, '%H'))

#make an empty column and apply the function
tws_merge['hour'] = np.NaN
tws_merge['hour'] = tws_merge.apply(process_time, axis=1)

In [10]:
map_hour = folium.Map(location=[43.651070, -79.347015], zoom_start=3) 

from folium.plugins import HeatMapWithTime
# List comprehension to make out list of lists
heat_data = [[[row['Latitude'],row['Lontitude']] for index, row in tws_merge[tws_merge['hour'] == i].iterrows()] for i in range(0,24)]

# Plot it on the map
hm = HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_hour)

map_hour


In [13]:
map_hour.save('corono_50_tweets_time_heat_map.html')